In [1]:
library(ggplot2)
library(scatterplot3d)
library(tidyverse)
library(reshape2)

-- Attaching core tidyverse packages ---------------------------------------------------------------- tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v lubridate 1.9.3     v tibble    3.2.1
v purrr     1.0.2     v tidyr     1.3.1
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: 'reshape2'


The following object is masked from 'package:tidyr':

    smiths




In [2]:
setwd('C:/BMK_new_drive/WorkFlow_test')

In [13]:
args <- c('5th_132indv', 'eigenval', 'eigenvec')

prefix <- args[1]
evalue <- paste(prefix, '.', args[2], sep='')
evector <- paste(prefix, '.', args[3], sep='')

eigenval <- scan(evalue)
eigenvec <- read_table2(evector, col_names=FALSE)
eigenvec <- eigenvec[, -1]
names(eigenvec)[1] <- 'ind'
names(eigenvec)[2:ncol(eigenvec)] <- paste('PC', 1: (ncol(eigenvec) - 1), sep='')
# names(eigenvec)
ts <- data.frame(PC=1:(ncol(eigenvec)-1), pve=eigenval/sum(eigenval)*100)
# ggplot(ts, aes(PC, pve)) + geom_bar(stat='identity') + ylab('pct') + coord_equal()
graphics.off()
pdf('test3d.pdf', width=10, height=10)
scatterplot3d(eigenvec[, 2:4], type='h', pch=16, color='red')
# eigenvec
dev.off()


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  .default = col_double(),
  X1 = col_character(),
  X2 = col_character()
)
i Use `spec()` for the full column specifications.

Warning message:
"Unknown or uninitialised column: `color`."


png 
  2

In [2]:
# vcf统计量测试用例
# args[1] = path, args[2] = prefix, args[3].... = extension of these statistic files

# args <- commandArgs(trailingOnly=TRUE)
args <- c('5th_132indv', 'frq', 'idepth', 'ldepth.mean', 'lqual', 'imiss', 'lmiss', 'het')

alle_freq <- paste(args[2], '.', args[3], sep='')
indv_mean_depth <- paste(args[2], '.', args[4], sep='')
site_mean_depth <- paste(args[2], '.', args[5], sep='')
site_quality <- paste(args[2], '.', args[6], sep='')
missing_data_per_indv <- paste(args[2], '.', args[7], sep='')
missing_data_per_site <- paste(args[2], '.', args[8], sep='')
heterozygosity <- paste(args[2], '.', args[9], sep='')

draw_lst <- c(site_quality, site_mean_depth, indv_mean_depth, missing_data_per_indv, missing_data_per_site, alle_freq, heterozygosity)

#Site quality
qual_data <- read_delim(draw_lst[1], col_names=c('chrom', 'pos', 'qual'), skip=1)
p1 <- ggplot(qual_data, aes(qual)) + geom_density(fill='dodgerblue1', colour='blue', alpha=0.3)
p1 <- p1 + theme(panel.grid.major=element_blank(), panel.grid.minor=element_blank(), panel.background=element_rect(fill='transparent'), axis.line=element_line(colour='black'), axis.text=element_text(size=14), axis.title=element_text(size=20)) + xlim(0, 10000)
ggsave('site_qual.pdf', p1, scale=3, dpi=300)

#Site mean depth
sdepth_data <- read_delim(draw_lst[2], col_names=c("chr", "pos", "mean_depth", "var_depth"), skip=1)
p2 <- ggplot(sdepth_data, aes(mean_depth)) + geom_density(fill='dodgerblue1', colour='blue', alpha=0.3)
p2 <- p2 + theme(panel.grid.major=element_blank(), panel.grid.minor=element_blank(), panel.background=element_rect(fill='transparent'), axis.line=element_line(colour='black'), axis.text=element_text(size=14), axis.title=element_text(size=20)) + xlim(0, 50)
ggsave('site_depth.pdf', p2, scale=3, dpi=300)

#indv mean depth
idepth_data <- read_delim(draw_lst[3], col_names=c("ind", "nsites", "depth"), skip=1)
p3 <- ggplot(idepth_data, aes(depth)) + geom_histogram(fill='dodgerblue1', colour='blue', alpha=0.3)
p3 <- p3 + theme(panel.grid.major=element_blank(), panel.grid.minor=element_blank(), panel.background=element_rect(fill='transparent'), axis.line=element_line(colour='black'), axis.text=element_text(size=14), axis.title=element_text(size=20))
ggsave('individual_depth.pdf', p3, scale=3, dpi=300)

#missing data per indv
imiss_data <- read_delim(draw_lst[4], col_names=c("ind", "ndata", "nfiltered", "nmiss", "fmiss"), skip=1)
p4 <- ggplot(imiss_data, aes(fmiss)) + geom_histogram(fill='dodgerblue1', colour='blue', alpha=0.3)
p4 <- p4 + theme(panel.grid.major=element_blank(), panel.grid.minor=element_blank(), panel.background=element_rect(fill='transparent'), axis.line=element_line(colour='black'), axis.text=element_text(size=14), axis.title=element_text(size=20))
ggsave('individual_missing_data.pdf', p4, scale=3, dpi=300)

#missing data per site
lmiss_data <- read_delim(draw_lst[5], col_names=c("ind", "ndata", "nfiltered", "nmiss", "fmiss"), skip=1)
p5 <- ggplot(lmiss_data, aes(fmiss)) + geom_histogram(fill='dodgerblue1', colour='blue', alpha=0.3)
p5 <- p5 + theme(panel.grid.major=element_blank(), panel.grid.minor=element_blank(), panel.background=element_rect(fill='transparent'), axis.line=element_line(colour='black'), axis.text=element_text(size=14), axis.title=element_text(size=20))
ggsave('site_missing_data.pdf', p5, scale=3, dpi=300)

#Allele freq
freq_data <- read_delim(draw_lst[6], delim='\t', col_names=c('chrom', 'pos', 'n_alleles', 'chr', 'a1', 'a2'), skip=1)
freq_data$min_allele <- freq_data %>% select(a1, a2) %>% apply(1, min)
p6 <- ggplot(freq_data, aes(min_allele)) + geom_density(fill='dodgerblue1', colour='blue', alpha=0.3)
p6 <- p6 + theme(panel.grid.major=element_blank(), panel.grid.minor=element_blank(), panel.background=element_rect(fill='transparent'), axis.line=element_line(colour='black'), axis.text=element_text(size=14), axis.title=element_text(size=20))
ggsave('allele_freq.pdf', p6, scale=3, dpi=300)

#heterozygosity
het_data <- read_delim(draw_lst[7], col_names=c("ind","ho", "he", "nsites", "f"), skip=1)
p7 <- ggplot(het_data, aes(f)) + geom_histogram(fill='dodgerblue1', colour='blue', alpha=0.3)
p7 <- p7 + theme(panel.grid.major=element_blank(), panel.grid.minor=element_blank(), panel.background=element_rect(fill='transparent'), axis.line=element_line(colour='black'), axis.text=element_text(size=14), axis.title=element_text(size=20))
ggsave('heterozygosity.pdf', p7, scale=3, dpi=300)